In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv()

client = OpenAI(api_key=os.environ['LLM_API_KEY'])

MODEL_NAME = "gpt-4o-mini"

In [ ]:
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='アンケート')

,アンケート
0,「使った瞬間、髪がしっとりサラサラになり驚きました。香りも強すぎずリラックスできる香りで気に...
1,「洗い上がりはスッキリするのに、髪が全然きしまず指通りが良くなりました！乾燥しがちな髪質です...
2,「香りが少し甘めで、好みが分かれそうです。個人的には好きですが、甘い香りが苦手な方には少し強...
3,「ボトルのデザインがシンプルでおしゃれなので、バスルームが映える感じがします。使用感も良くて...
4,「使い始めてから頭皮のかゆみが減りました。敏感肌なので合わないシャンプーが多いのですが、これ...


In [4]:
def extract_keywords(text):
    prompt_text = f"以下のアンケートから主要なキーワードをカンマ区切りで抽出してください。\nアンケート: {text}"

    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt_text},
            ],
            max_tokens=100,
            temperature=0.3
        )
        keywords = response.choices[0].message.content.strip()
        return keywords
    
    except Exception as e:
        print(f"API呼び出しでエラーが発生しました: {e}")
        return "エラー"


In [ ]:
df['キーワード'] = ""

for index, row in df.iterrows():
    survey_text = row['アンケート']
    
    keywords = extract_keywords(survey_text)
    df.at[index, 'キーワード'] = keywords

df.to_excel("アンケート分析結果.xlsx", index=False)

,アンケート,キーワード
0,「使った瞬間、髪がしっとりサラサラになり驚きました。香りも強すぎずリラックスできる香りで気に...,"髪, しっとり, サラサラ, 驚き, 香り, リラックス, 泡立ち, 少量, 洗える, コス..."
1,「洗い上がりはスッキリするのに、髪が全然きしまず指通りが良くなりました！乾燥しがちな髪質です...,"洗い上がり, スッキリ, 髪, きしまず, 指通り, 乾燥, 髪質, シャンプー, 翌朝, ..."
2,「香りが少し甘めで、好みが分かれそうです。個人的には好きですが、甘い香りが苦手な方には少し強...,"香り, 甘め, 好み, 強く感じる, 洗い上がり, 軽やか, 頭皮, すっきり, 夏場, 快適"
3,「ボトルのデザインがシンプルでおしゃれなので、バスルームが映える感じがします。使用感も良くて...,"ボトルデザイン, シンプル, おしゃれ, バスルーム, 使用感, シャンプー, 乾燥, パサ..."
4,「使い始めてから頭皮のかゆみが減りました。敏感肌なので合わないシャンプーが多いのですが、これ...,"頭皮のかゆみ, 敏感肌, シャンプー, 刺激が少ない, 使いやすい, 香り, ナチュラル, ..."
